# Graph Exploration Testing

In [ ]:
import importnb
import rdflib

from ipyradiant import FileManager, GetOutgoingPredicateObjects, PathLoader

with importnb.Notebook():
    try:
        from .Improved_Vis import iv
    except ImportError:
        from Improved_Vis import iv

## We will start by loading in the `InteractiveViewer` instance found in the `Improved_Vis.ipynb` notebook.

In [ ]:
iv

### We can now run manual tests to make sure the different features are working correctly.

We will start by testing the expanding node feature.

In [ ]:
# to start, get the correct amount of nodes that should be present when expanding upon Darth Vader
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
new_data = GetOutgoingPredicateObjects.run_query(
    graph=rdf_graph, subject="https://swapi.co/resource/human/4"
)

We know the correct amount of nodes after expanding upon darth vader should be 10, for
the 8 nodes from the expansion as well as Luke Skywalker, which was not connected
before. We can now manually expand on Darth Vader and make sure that's the case.

In [ ]:
for node in iv.cytoscape_widget.graph.nodes:
    if node.data["rdfs:label"] == "Darth Vader":
        iv.selected_node = node
iv.expand_button_clicked(None)
assert (
    len(iv.cytoscape_widget.graph.nodes) == 10
), "Improper amount of nodes in the graph after expansion."

We can now test the undo expansion function of the graph, which should return the graph
to only three nodes.

In [ ]:
iv.undo_expansion(None)
assert (
    len(iv.cytoscape_widget.graph.nodes) == 3
), "Improper amount of nodes in the graph after undoing expansion."

We now want to test the removal of all temporary nodes from the graph. In order to do
this we will expand on Darth Vader, make one of the temporary nodes permanent by
changing the class, and then remove all the temporary nodes to test that the new
permanent node still remains in the graph.

In [ ]:
# expand upon darth vader
for node in iv.cytoscape_widget.graph.nodes:
    if node.data.get("rdfs:label") == "Darth Vader":
        iv.selected_node = node
iv.expand_button_clicked(None)
# change class of one of the temp nodes, lets say return of the jedi
for node in iv.cytoscape_widget.graph.nodes:
    if str(node.data["_label"]) == "Return of the Jedi":
        node.classes = "clicked"
iv.remove_temp_nodes(None)
assert len(iv.cytoscape_widget.graph.nodes) == 4